# ДЗ №1

In [1]:
import pandas as pd
import numpy as np

In [68]:
recommended_list   = [143,  32, 1134, 991,  27, 1543, 3345,  533,   11,  521] #id товаров
prices_recommended = [100, 200,  300, 400, 500, 1000, 1000, 1000, 1000, 1000]
bought_list   = [ 521,  32, 143, 991]
prices_bought = [1000, 200, 100, 400]

## 1. Hit rate

In [61]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    flags = np.isin(bought_list, recommended_list)
    hit_rate = (flags.sum() > 0) * 1
    return hit_rate

In [62]:
hit_rate_at_k(recommended_list, bought_list, 5)

1

## 2. Precision

In [63]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    # Лучше считать через скалярное произведение, а не цикл
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])

    flags = np.isin(recommended_list, bought_list) * 1
    
    return  (flags * prices_recommended).sum() / prices_recommended.sum()

In [64]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, 5)

0.4666666666666667

## 3. Recall

In [80]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])   
    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommended_list, bought_list) * 1
    revenue_recommended = (flags * prices_recommended).sum()
    
    return revenue_recommended / prices_bought.sum()

In [81]:
recall_at_k(recommended_list, bought_list, 5)

0.75

In [82]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, 5)

0.4117647058823529

---

# Метрики ранжирования

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [109]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1):
        
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [110]:
ap_k(recommended_list, bought_list, k=5)

0.41666666666666663

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

> **не совсем понял где брать кол-во юзеров в данном случае ? поэтому сделал как понял**

In [114]:
def map_k(recommended_list, users, k=5):
    
    result = 0
    for bought_list in users: # ?????
        result += ap_k(recommended_list, bought_list, k) # ??????
    
    return result / len(users)

> **не совсем понял где брать кол-во юзеров в данном случае ? поэтому сделал как понял**

In [113]:
map_k(recommended_list, [
    bought_list, 
    [27, 32]
])

0.2708333333333333

## MRR@k

In [106]:
def reciprocal_rank(recommended_list, bought_list):
    flags = np.isin(recommended_list, bought_list)
    rank = np.where(flags == True)[0]
    return 1/(rank[0]+1) if len(rank) else 0

In [107]:
reciprocal_rank(recommended_list, bought_list)

1.0

In [108]:
reciprocal_rank(recommended_list, [27])

0.2